# Audio Processing

This notebook plays with different audio procesing algorithms and develops a flexible way to wire them up.

The problem: ad-hoc implementations of processing algorithms mix I/O and algorithm logic, making the algorithm less portable.

The solution: define standard patterns for algorithms to declare inputs and outputs and specify how data moves from inputs to outputs, and for clients to create and join algorithms into a full processing graph.

Use cases:
- Recording a song
- Creating a visualization that can analyze audio files or live inputs
- Creating a real-time audio effect
- Creating an offline audio effect

# Design

The data flow building block is the Channel, a one-way 1-N data transport. 

Channel output can be connected to buffers or transform methods. Channels have a write method for input.

Algorithms use one channel per logical input and output. They configure the output side of their input channels, input side of their output channels using buffers or transform objects. They are also responsible for scheduling work to read and write any buffers used to connect inputs and outputs.

In [1]:
%matplotlib widget

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import pyaudio
import numpy as np
from IPython.display import display
import ipywidgets as widgets
from pprint import pprint
import logging
import threading
import librosa
import time

from jupyter_utils import *
from audio_tools import Daw

# Set up the root logger and display its output in this cell.
logger = get_cell_logger()
logger.show_logs()

Button(description='Clear logs', style=ButtonStyle())

Output(layout=Layout(border='1px solid white', height='300px', overflow='auto'))

In [2]:
try:
    daw.terminate()
except NameError:
    pass

fig, ax = start_figure("DAW Monitor", figsize=(8, 5))
daw = Daw()

# ax.plot(range(10))
# plt.show()


FigureCanvasNbAgg()

Button(description='Start', style=ButtonStyle())

Button(description='Terminate', style=ButtonStyle())

Button(description='Clear logs', style=ButtonStyle())

Output(layout=Layout(border='1px solid white', height='300px', overflow='auto'))

Output(layout=Layout(border='1px solid white'))

In [3]:
4096 / 44100

0.09287981859410431